In [2]:
import torch

In [3]:
bin(8)

'0b1000'

In [14]:
def encode_year(year: int) -> torch.Tensor:
    # We are ok with years 0-4095
    VLEN = 12
    bitstr = bin(year)[2:]
    v = torch.tensor(
        [int(bit) for bit in bin(year)[2:]],
        dtype=torch.float32
    )
    if len(bitstr) < VLEN:
        return torch.cat([
            torch.zeros(VLEN - len(bitstr)),
            v
        ])
    else:
        return v

In [17]:
encode_year(4)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])